# Analyze and tidy-up Arivale inter-omic interaction analysis results

Inputs:
* 240427_FE2_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_FE4_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_Fhealth_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_Funhealth_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_ME2_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_ME4_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_Mhealth_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240427_Munhealth_interomic_interact_Arivale_5SD_winsor.txt.gz  
* 240504_E2_allele_dose_interomic_interaction_5SD_winsor.txt.gz  
* 240504_E4_allele_dose_interomic_interaction_5SD_winsor.txt.gz  
* 240504_contin_del_age_interomic_interaction_5SD_winsor.txt.gz  

Outputs: data for Supplementary File 3; data for Table 2 and Supplementary Table 4

'Save' lines are commented out  
'Healthy' and 'Unhealthy' are renamed as 'Bio_Young' and 'Bio_Old' in figures and data files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
import math
import statsmodels

In [2]:
rename_chems_dict_temp = {'Urate' : 'URIC ACID',
 'Insulin' : 'INSULIN',
 'HbA1c' : 'GLYCOHEMOGLOBIN A1C',
 'Creatinine' : 'CREATININE ENZ, SER',
 'Triglycerides' : 'TRIGLYCERIDES',
 'Urea' : 'UREA NITROGEN',
 'hs_CRP' : 'CRP HIGH SENSITIVITY',
 'Glucose' : 'GLUCOSE',
 'Haemoglobin' : 'HEMOGLOBIN',
 'eGFR' : 'GFR, MDRD',
 'Tot-cholesterol' : 'CHOLESTEROL, TOTAL',
 'HDL' : 'HDL CHOL DIRECT',
 'LDL' : 'LDL-CHOL CALCULATION'}

rename_chems_dict = {value: key for key, value in rename_chems_dict_temp.items()}

In [3]:
def get_pairs_to_validate(df):
    
    validation_list = []
    
    for i in df.index:
        if df.type1[i] == 'clinical' and df.type2[i] == 'metabolite':
            addition1_temp, addition2 = df.col1[i], df.name2[i]
            
            if addition1_temp in rename_chems_dict:
                addition1 = rename_chems_dict[addition1_temp]
            
                validation_list.append((addition1, addition2))   
    
    
    return validation_list

# load in result files from Code02

In [4]:
FE2_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_FE2_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
FE4_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_FE4_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
ME2_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_ME2_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
ME4_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_ME4_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')

In [5]:
# perform missing pFDR calculation, typo
np.seterr(all='warn')
(adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(ME4_compare_inter.loc[~ME4_compare_inter['analyte2:C(APOE_Status)[T.E4]_p'].isnull(), 'analyte2:C(APOE_Status)[T.E4]_p'], alpha=0.05, method='fdr_bh')
ME4_compare_inter.loc[~ME4_compare_inter['analyte2:C(APOE_Status)[T.E4]_p'].isnull(), 'pval_adj'] = adj_pval
ME4_compare_inter.sort_values(['pval_adj'], ascending=True, inplace=True)

In [6]:
F_health_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_Fhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
F_unhealth_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_Funhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
M_health_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_Mhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')
M_unhealth_compare_inter = pd.read_table('/notebooks/0. APOE-Multiomics/Data_Files/240427_Munhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', sep='\t', compression='gzip')

# save csv of each for pairs with interaction beta pre-adjusted pval < 0.05
# generate a list of pairs with interaction pFDR < 0.1 to specifically test/validate in TwinsUK

In [7]:
FE2_output = FE2_compare_inter[(FE2_compare_inter.pval_adj < 0.1)|(FE2_compare_inter['analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'] < 0.05)]
# FE2_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_FE2_Arivale_5SD_interact.csv')

In [8]:
FE2_output2 = FE2_output[(FE2_output.pval_adj < 0.1)]
FE2_validation_list = get_pairs_to_validate(FE2_output2)

In [9]:
FE2_validation_list

[]

In [10]:
ME2_output = ME2_compare_inter[(ME2_compare_inter.pval_adj < 0.1)|(ME2_compare_inter['analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'] < 0.05)]
# ME2_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_ME2_Arivale_5SD_interact.csv')

In [11]:
ME2_output2 = ME2_output[(ME2_output.pval_adj < 0.1)]
ME2_validation_list = get_pairs_to_validate(ME2_output2)

In [12]:
ME2_validation_list

[('Triglycerides', 'ribitol'),
 ('HbA1c', 'phenol sulfate'),
 ('Triglycerides', 'ceramide (d18:1/20:0, d16:1/22:0, d20:1/18:0)*'),
 ('Glucose', 'phenol sulfate')]

In [13]:
FE4_output = FE4_compare_inter[(FE4_compare_inter.pval_adj < 0.1)|(FE4_compare_inter['analyte2:C(APOE_Status)[T.E4]_p'] < 0.05)]
# FE4_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_FE4_Arivale_5SD_interact.csv')

In [14]:
FE4_output2 = FE4_output[(FE4_output.pval_adj < 0.1)]
FE4_validation_list = get_pairs_to_validate(FE4_output2)

In [15]:
FE4_validation_list

[]

In [16]:
ME4_output = ME4_compare_inter[(ME4_compare_inter.pval_adj < 0.1)|(ME4_compare_inter['analyte2:C(APOE_Status)[T.E4]_p'] < 0.05)]
# ME4_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_ME4_Arivale_5SD_interact.csv')

In [17]:
ME4_output2 = ME4_output[(ME4_output.pval_adj < 0.1)]
ME4_validation_list = get_pairs_to_validate(ME4_output2)

In [18]:
ME4_validation_list

[]

In [19]:
Fhealth_output = F_health_compare_inter[(F_health_compare_inter.pval_adj < 0.1)|(F_health_compare_inter['analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'] < 0.05)]
# Fhealth_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_Fhealth_Arivale_5SD_interact.csv')

In [20]:
Fhealth_output2 = Fhealth_output[(Fhealth_output.pval_adj < 0.1)]
Fhealth_validation_list = get_pairs_to_validate(Fhealth_output2)

In [21]:
Fhealth_validation_list

[]

In [22]:
Mhealth_output = M_health_compare_inter[(M_health_compare_inter.pval_adj < 0.1)|(M_health_compare_inter['analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'] < 0.05)]
# Mhealth_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_Mhealth_Arivale_5SD_interact.csv')

In [23]:
Mhealth_output2 = Mhealth_output[(Mhealth_output.pval_adj < 0.1)]
Mhealth_validation_list = get_pairs_to_validate(Mhealth_output2)

In [24]:
Mhealth_validation_list

[]

In [25]:
Funhealth_output = F_unhealth_compare_inter[(F_unhealth_compare_inter.pval_adj < 0.1)|(F_unhealth_compare_inter['analyte2:C(Model_Health)[T.Unhealthy]_p'] < 0.05)]
# Funhealth_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_Funhealth_Arivale_5SD_interact.csv')

In [26]:
Funhealth_output2 = Funhealth_output[(Funhealth_output.pval_adj < 0.1)]
Funhealth_validation_list = get_pairs_to_validate(Funhealth_output2)

In [27]:
Funhealth_validation_list

[('HbA1c', 'alpha-ketobutyrate'),
 ('HbA1c', 'gluconate'),
 ('HbA1c', '1-(1-enyl-palmitoyl)-2-myristoyl-GPC (P-16:0/14:0)*'),
 ('HbA1c', 'X - 19438'),
 ('HbA1c', 'taurine'),
 ('HbA1c', '2-hydroxybutyrate/2-hydroxyisobutyrate'),
 ('HbA1c', 'X - 24295'),
 ('HbA1c', 'alpha-ketoglutarate'),
 ('HbA1c', '4-methyl-2-oxopentanoate'),
 ('HbA1c', 'fructose'),
 ('Glucose', 'gluconate'),
 ('Glucose', '1-(1-enyl-palmitoyl)-2-myristoyl-GPC (P-16:0/14:0)*'),
 ('HbA1c', 'N-acetylleucine'),
 ('HbA1c', '3-methyl-2-oxovalerate'),
 ('HbA1c', 'pyruvate'),
 ('HbA1c', 'margarate (17:0)'),
 ('Glucose', 'alpha-ketobutyrate'),
 ('HbA1c', '3-hydroxy-2-ethylpropionate'),
 ('HbA1c', 'mannose'),
 ('Urea', 'linolenoylcarnitine (C18:3)*'),
 ('Glucose', 'X - 16087')]

In [28]:
Munhealth_output = M_unhealth_compare_inter[(M_unhealth_compare_inter.pval_adj < 0.1)|(M_unhealth_compare_inter['analyte2:C(Model_Health)[T.Unhealthy]_p'] < 0.05)]
# Munhealth_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_Munhealth_Arivale_5SD_interact.csv')

In [29]:
Munhealth_output2 = Munhealth_output[(Munhealth_output.pval_adj < 0.1)]
Munhealth_validation_list = get_pairs_to_validate(Munhealth_output2)

In [30]:
Munhealth_validation_list

[('Glucose', 'N-acetylvaline'),
 ('Glucose', 'linoleoyl-linoleoyl-glycerol (18:2/18:2) [1]*'),
 ('Glucose', 'mannose'),
 ('HbA1c', 'N-acetylvaline'),
 ('Glucose', 'glutamate'),
 ('HbA1c', 'linoleoyl-linoleoyl-glycerol (18:2/18:2) [1]*'),
 ('HbA1c', 'aspartate'),
 ('Glucose', 'palmitoleoyl-linoleoyl-glycerol (16:1/18:2) [1]*'),
 ('HbA1c', 'pyruvate'),
 ('Glucose', 'pyruvate'),
 ('Glucose', 'leucine'),
 ('Glucose', 'aspartate'),
 ('Glucose', 'palmitoyl-linoleoyl-glycerol (16:0/18:2) [2]*'),
 ('Haemoglobin', 'N-acetylthreonine'),
 ('HbA1c', 'glutamate'),
 ('Haemoglobin', 'gamma-glutamylglycine'),
 ('HbA1c', '1-carboxyethylvaline'),
 ('HbA1c', 'linoleoyl-arachidonoyl-glycerol (18:2/20:4) [1]*'),
 ('Glucose', 'oleoyl-linoleoyl-glycerol (18:1/18:2) [1]'),
 ('HbA1c', 'isoleucine'),
 ('Glucose', 'cortolone glucuronide (1)'),
 ('HbA1c', 'leucine'),
 ('Glucose', 'butyrylcarnitine (C4)'),
 ('Glucose', 'phenol sulfate'),
 ('Glucose', 'linoleoyl-arachidonoyl-glycerol (18:2/20:4) [1]*'),
 ('HbA1c', 

---

# analyze allele dose & continuous delta age interactions

In [31]:
E2_df = pd.read_table('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/240504_E2_allele_dose_interomic_interaction_5SD_winsor.txt.gz', sep='\t', compression='gzip')
E4_df = pd.read_table('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/240504_E4_allele_dose_interomic_interaction_5SD_winsor.txt.gz', sep='\t', compression='gzip')
del_age_continous_df = pd.read_table('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/240504_contin_del_age_interomic_interaction_5SD_winsor.txt.gz', sep='\t', compression='gzip')

/opt/conda/envs/arivale-py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,3,4,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
E2_df_output = E2_df[(E2_df.E2_pFDR < 0.1)]

In [33]:
E2_allele_validation_list = get_pairs_to_validate(E2_df_output)

In [34]:
E2_allele_validation_list

[('HbA1c', 'malate'),
 ('HbA1c', 'fumarate'),
 ('Glucose', 'aconitate [cis or trans]'),
 ('Glucose', '3-hydroxy-2-ethylpropionate'),
 ('Tot-cholesterol', 'arabonate/xylonate')]

In [35]:
# E2_df_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_E2_Arivale_5SD_allele_dose_interact_only_pFDR.csv')

In [36]:
E4_df_output = E4_df[(E4_df.E4_pFDR < 0.1)]

In [37]:
E4_allele_validation_list = get_pairs_to_validate(E4_df_output)

In [38]:
# E4_df_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_E4_Arivale_5SD_allele_dose_interact_pFDR_only.csv')

In [39]:
E4_allele_validation_list

[]

In [40]:
delta_age_df_output = del_age_continous_df[(del_age_continous_df.delta_age_pFDR < 0.1)]

In [41]:
delta_age_validation_list = get_pairs_to_validate(delta_age_df_output)

In [42]:
# delta_age_df_output.to_csv('/notebooks/Final_Paper/240423_Investigate_Redo_Interactions/paper_output_table/240625_delta_age_Arivale_5SD_continuous_interact_pFDR_only.csv')

In [43]:
delta_age_validation_list

[('Insulin', 'eicosenoate (20:1)'),
 ('Tot-cholesterol', 'alpha-hydroxyisocaproate'),
 ('Tot-cholesterol', 'sphingomyelin (d18:2/23:1)*'),
 ('Tot-cholesterol', 'leucine'),
 ('Tot-cholesterol', '1-palmitoyl-GPE (16:0)'),
 ('Glucose', 'palmitate (16:0)'),
 ('Glucose', 'oleoyl-arachidonoyl-glycerol (18:1/20:4) [2]*'),
 ('Glucose', '1-carboxyethylleucine'),
 ('Glucose', '3,5-dichloro-2,6-dihydroxybenzoic acid'),
 ('Glucose', 'X - 23639'),
 ('Glucose', 'palmitoyl-oleoyl-glycerol (16:0/18:1) [2]*'),
 ('Glucose', 'beta-cryptoxanthin'),
 ('Glucose', '1-stearoyl-2-docosapentaenoyl-GPC (18:0/22:5n6)*'),
 ('Glucose', '1-eicosenoyl-GPC (20:1)*'),
 ('Glucose', 'valine'),
 ('Glucose', 'arginine'),
 ('Glucose', '1-oleoyl-GPC (18:1)'),
 ('Glucose', 'X - 24337'),
 ('Glucose', 'dihomo-linoleate (20:2n6)'),
 ('Glucose', 'glutamine conjugate of C6H10O2 (1)*'),
 ('Glucose', '1-(1-enyl-palmitoyl)-GPC (P-16:0)*'),
 ('Glucose', 'glutamine'),
 ('Glucose', 'behenoyl dihydrosphingomyelin (d18:0/22:0)*'),
 ('Gluc

---

# find concordant/discordant interaction pairs between groups

In [44]:
FE2_sum_comp = FE2_compare_inter[(FE2_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(APOE_Status, Treatment(reference=1))[T.E2]', 'analyte2',
       'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]', 'pval_adj']]

FE2_sum_comp.rename(columns = {'C(APOE_Status, Treatment(reference=1))[T.E2]':'FE2_C(APOE_Status, Treatment(reference=1))[T.E2]',
                               'analyte2':'FE2_analyte2',
                               'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]':'FE2_interaction',
                              'pval_adj':'FE2_pval_adj'}
                    , inplace = True)

In [45]:
FE4_sum_comp = FE4_compare_inter[(FE4_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(APOE_Status)[T.E4]', 'analyte2',
       'analyte2:C(APOE_Status)[T.E4]', 'pval_adj']]

FE4_sum_comp.rename(columns = {'C(APOE_Status)[T.E4]':'FE4_C(APOE_Status)[T.E4]',
                               'analyte2':'FE4_analyte2',
                               'analyte2:C(APOE_Status)[T.E4]':'FE4_interaction',
                              'pval_adj':'FE4_pval_adj'}
                    , inplace = True)

In [46]:
ME4_sum_comp = ME4_compare_inter[(ME4_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(APOE_Status)[T.E4]', 'analyte2',
       'analyte2:C(APOE_Status)[T.E4]', 'pval_adj']]

ME4_sum_comp.rename(columns = {'C(APOE_Status)[T.E4]':'ME4_C(APOE_Status)[T.E4]',
                               'analyte2':'ME4_analyte2',
                               'analyte2:C(APOE_Status)[T.E4]':'ME4_interaction',
                              'pval_adj':'ME4_pval_adj'}
                    , inplace = True)

In [47]:
ME2_sum_comp = ME2_compare_inter[(ME2_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(APOE_Status, Treatment(reference=1))[T.E2]', 'analyte2', 'C(APOE_Status, Treatment(reference=1))[T.E2]_p', 'analyte2_p',
       'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]', 'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p', 'pval_adj']]

ME2_sum_comp.rename(columns = {'C(APOE_Status, Treatment(reference=1))[T.E2]':'ME2_C(APOE_Status, Treatment(reference=1))[T.E2]',
                               'analyte2':'ME2_analyte2',
                               'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]':'ME2_interaction',
                              'pval_adj':'ME2_pval_adj'}
                    , inplace = True)

In [48]:
Mhealth_sum_comp = M_health_compare_inter[(M_health_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(Model_Health, Treatment(reference=1))[T.Healthy]', 'analyte2',
       'analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]', 'pval_adj']]

Mhealth_sum_comp.rename(columns = {'C(Model_Health, Treatment(reference=1))[T.Healthy]':'Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]',
                               'analyte2':'Mhealth_analyte2',
                               'analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]':'Mhealth_interaction',
                              'pval_adj':'Mhealth_pval_adj'}
                    , inplace = True)

In [49]:
Fhealth_sum_comp = F_health_compare_inter[(F_health_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(Model_Health, Treatment(reference=1))[T.Healthy]', 'analyte2',
       'analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]', 'pval_adj']]

Fhealth_sum_comp.rename(columns = {'C(Model_Health, Treatment(reference=1))[T.Healthy]':'Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]',
                               'analyte2':'Fhealth_analyte2',
                               'analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]':'Fhealth_interaction',
                              'pval_adj':'Fhealth_pval_adj'}
                    , inplace = True)

In [50]:
Funhealth_sum_comp = F_unhealth_compare_inter[(F_unhealth_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(Model_Health)[T.Unhealthy]', 'analyte2',
       'analyte2:C(Model_Health)[T.Unhealthy]', 'pval_adj']]

Funhealth_sum_comp.rename(columns = {'C(Model_Health)[T.Unhealthy]':'Funhealth_C(Model_Health)[T.Unhealthy]',
                               'analyte2':'Funhealth_analyte2',
                               'analyte2:C(Model_Health)[T.Unhealthy]':'Funhealth_interaction',
                              'pval_adj':'Funhealth_pval_adj'}
                    , inplace = True)

In [51]:
Munhealth_sum_comp = M_unhealth_compare_inter[(M_unhealth_compare_inter.pval_adj < 0.1)][['col1', 'name1', 'col2', 'name2',
       'C(Model_Health)[T.Unhealthy]', 'analyte2',
       'analyte2:C(Model_Health)[T.Unhealthy]', 'pval_adj']]

Munhealth_sum_comp.rename(columns = {'C(Model_Health)[T.Unhealthy]':'Munhealth_C(Model_Health)[T.Unhealthy]',
                               'analyte2':'Munhealth_analyte2',
                               'analyte2:C(Model_Health)[T.Unhealthy]':'Munhealth_interaction',
                              'pval_adj':'Munhealth_pval_adj'}
                    , inplace = True)

In [52]:
pd.merge(ME2_sum_comp, ME4_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.ME4_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,interact_dif_sign


In [53]:
pd.merge(ME2_sum_comp, FE2_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.FE2_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,interact_dif_sign


In [54]:
pd.merge(ME2_sum_comp, FE4_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.FE4_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,interact_dif_sign


In [55]:
pd.merge(ME2_sum_comp, Mhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.Mhealth_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,interact_dif_sign


In [56]:
pd.merge(ME2_sum_comp, Munhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.Munhealth_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,interact_dif_sign
0,100020205,hydroxyasparagine**,Megasphaera,Megasphaera,0.340723,0.004499,1.795260e-07,0.432573,0.116053,1.447671e-07,0.007268,0.185362,0.000957,0.058168,0.041052,1.0
1,CVD2_P19883,FST,181,laurate (12:0),-0.410828,-0.063484,2.015443e-03,0.107451,0.596664,2.550041e-07,0.009991,-0.240699,0.026790,0.450103,0.014488,1.0
2,GLYCOHEMOGLOBIN A1C,Hemoglobin A1C,100001510,phenol sulfate,-0.064988,-0.006159,3.165302e-04,0.123647,0.040708,9.506196e-07,0.024210,-0.004354,-0.008611,0.028598,0.040497,1.0
3,GLUCOSE,Glucose,100001510,phenol sulfate,-0.109940,-0.010390,4.155501e-04,0.132385,0.064965,5.832351e-06,0.074896,-0.017895,-0.017410,0.051070,0.036834,1.0


In [57]:
pd.merge(ME2_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign


In [58]:
pd.merge(ME2_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME2_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,"ME2_C(APOE_Status, Treatment(reference=1))[T.E2]",ME2_analyte2,"C(APOE_Status, Treatment(reference=1))[T.E2]_p",analyte2_p,ME2_interaction,"analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p",ME2_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign


In [59]:
pd.merge(ME4_sum_comp, FE2_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.FE2_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,interact_dif_sign


In [60]:
pd.merge(ME4_sum_comp, FE4_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.FE4_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,interact_dif_sign


In [61]:
pd.merge(ME4_sum_comp, Mhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.Mhealth_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,interact_dif_sign


In [62]:
pd.merge(ME4_sum_comp, Munhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.Munhealth_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,interact_dif_sign


In [63]:
pd.merge(ME4_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign


In [64]:
pd.merge(ME4_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.ME4_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,ME4_C(APOE_Status)[T.E4],ME4_analyte2,ME4_interaction,ME4_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign


In [65]:
pd.merge(FE2_sum_comp, FE4_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE2_interaction * x.FE4_interaction))

,col1,name1,col2,name2,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,interact_dif_sign


In [66]:
pd.merge(FE2_sum_comp, Mhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE2_interaction * x.Mhealth_interaction))

,col1,name1,col2,name2,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,interact_dif_sign


In [67]:
pd.merge(FE2_sum_comp, Munhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE2_interaction * x.Munhealth_interaction))

,col1,name1,col2,name2,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,interact_dif_sign


In [68]:
pd.merge(FE2_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE2_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign


In [69]:
pd.merge(FE2_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE2_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,"FE2_C(APOE_Status, Treatment(reference=1))[T.E2]",FE2_analyte2,FE2_interaction,FE2_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign


In [70]:
pd.merge(FE4_sum_comp, Mhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE4_interaction * x.Mhealth_interaction))

,col1,name1,col2,name2,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,interact_dif_sign


In [71]:
pd.merge(FE4_sum_comp, Munhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE4_interaction * x.Munhealth_interaction))

,col1,name1,col2,name2,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,interact_dif_sign


In [72]:
pd.merge(FE4_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE4_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign
0,100002488,isoursodeoxycholate,Prevotella_2,Prevotella_2,-0.65642,0.270759,-0.307116,0.081189,1.335885,0.087764,0.572191,0.036384,-1.0


In [73]:
pd.merge(FE4_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.FE4_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,FE4_C(APOE_Status)[T.E4],FE4_analyte2,FE4_interaction,FE4_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign


In [74]:
pd.merge(Mhealth_sum_comp, Munhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Mhealth_interaction * x.Munhealth_interaction))

,col1,name1,col2,name2,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,interact_dif_sign


In [75]:
pd.merge(Mhealth_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Mhealth_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign


In [76]:
pd.merge(Mhealth_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Mhealth_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,"Mhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Mhealth_analyte2,Mhealth_interaction,Mhealth_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign


In [77]:
pd.merge(Munhealth_sum_comp, Fhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Munhealth_interaction * x.Fhealth_interaction))

,col1,name1,col2,name2,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,interact_dif_sign


In [78]:
pd.merge(Munhealth_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Munhealth_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,Munhealth_C(Model_Health)[T.Unhealthy],Munhealth_analyte2,Munhealth_interaction,Munhealth_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign
0,GLYCOHEMOGLOBIN A1C,Hemoglobin A1C,823,pyruvate,-0.119017,0.064594,0.140201,0.011761,-0.074477,0.086591,0.091230,0.066523,1.0
1,GLUCOSE,Glucose,INF_P14210,HGF,-1.079395,0.031724,0.128591,0.025697,-0.828725,0.020464,0.098898,0.091675,1.0
2,GLUCOSE,Glucose,999947988,X - 16087,-0.035197,0.011669,0.076780,0.060493,-0.023455,0.010904,0.076830,0.094191,1.0
3,GLYCOHEMOGLOBIN A1C,Hemoglobin A1C,803,mannose,-0.098301,0.086271,0.104592,0.069383,-0.069330,0.109138,0.073402,0.091675,1.0
4,GLUCOSE,Glucose,CVD3_Q86VB7,CD163,-0.645722,0.013946,0.088995,0.079372,-0.567551,0.015204,0.077090,0.062030,1.0


In [79]:
pd.merge(Fhealth_sum_comp, Funhealth_sum_comp, how='inner', on=['col1', 'col2', 'name1','name2']).assign(interact_dif_sign=lambda x: np.sign(x.Fhealth_interaction * x.Funhealth_interaction))

,col1,name1,col2,name2,"Fhealth_C(Model_Health, Treatment(reference=1))[T.Healthy]",Fhealth_analyte2,Fhealth_interaction,Fhealth_pval_adj,Funhealth_C(Model_Health)[T.Unhealthy],Funhealth_analyte2,Funhealth_interaction,Funhealth_pval_adj,interact_dif_sign
